In [19]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

In [2]:
SOURCE_FILE_NAME = '/home/sjb/Projects/Research/LinkedIn_OB/data/covariates/company_level_info.csv'

In [3]:
company_df = pd.read_csv(SOURCE_FILE_NAME)

In [4]:
company_industry_df = company_df['company_industry'].value_counts().reset_index().copy()

In [5]:
company_industry_df.columns = ['industry', 'count']

#### Check1 - Every name ends with ' industry'

In [6]:
(~company_industry_df['industry'].str.endswith(' industry')).sum()

0

In [7]:
company_industry_df['industry_short_name'] = company_industry_df['industry'].str.replace(' industry', '')
company_industry_df['industry_short_name_lower'] = company_industry_df['industry_short_name'].str.lower()

In [9]:
# company_industry_df['count'].hist(bins=50, )

In [10]:
# company_industry_df['count'].hist(range=(0, 20), bins=50)

## Map to GICS Sector

In [26]:
from fuzzywuzzy import fuzz

In [ ]:
fuzz.ratio()

In [12]:
gics_sector_df = pd.read_pickle('./gics_sector_mapping.pkl')

In [57]:
gics_desc_level1_map = gics_sector_df.set_index('desc_lower')['level1_desc_lower'].to_dict()

In [17]:
gics_uniq_sectors = gics_sector_df['desc_lower'].unique()

In [40]:
def get_match_candidates(ref, compare_list):
    
    if ref in compare_list:
        exact_match = True
    else:
        exact_match = False
        
    fuzz_ratio_scores = np.array([fuzz.ratio(ref, c) for c in compare_list])
    max_fuzz_ratio_score = fuzz_ratio_scores.max()
    max_fuzz_ratio_idxes = np.where(fuzz_ratio_scores == max_fuzz_ratio_score)[0]
    fuzz_ratio_candidates = [compare_list[i] for i in max_fuzz_ratio_idxes]
    
    fuzz_partial_ratio_scores = np.array([fuzz.partial_ratio(ref, c) for c in compare_list])
    max_fuzz_partial_ratio_score = fuzz_partial_ratio_scores.max()
    max_fuzz_partial_ratio_idxes = np.where(fuzz_partial_ratio_scores == max_fuzz_partial_ratio_score)[0]
    fuzz_partial_ratio_candidates = [compare_list[i] for i in max_fuzz_partial_ratio_idxes]
        
    result = pd.Series({
        'exact_match': exact_match,
        'max_fuzz_ratio_score': max_fuzz_ratio_score,
        'fuzz_ratio_candidates': fuzz_ratio_candidates,
        'max_fuzz_partial_ratio_score': max_fuzz_partial_ratio_score,
        'fuzz_partial_ratio_candidates': fuzz_partial_ratio_candidates,
    })
    return result

In [41]:
fuzz_ratio_scores = np.array([fuzz.ratio('Financial Services', c) for c in gics_uniq_sectors])

In [42]:
np.where(fuzz_ratio_scores == fuzz_ratio_scores.max())[0]

array([75])

In [43]:
company_industry_df

,industry,count,industry_short_name,industry_short_name_lower
0,Higher Education industry,212,Higher Education,higher education
1,Retail industry,94,Retail,retail
2,Financial Services industry,84,Financial Services,financial services
3,Information Technology and Services industry,79,Information Technology and Services,information technology and services
4,Hospital & Health Care industry,64,Hospital & Health Care,hospital & health care
...,...,...,...,...
103,Graphic Design industry,1,Graphic Design,graphic design
104,Law Enforcement industry,1,Law Enforcement,law enforcement
105,Wireless industry,1,Wireless,wireless
106,"Glass, Ceramics & Concrete industry",1,"Glass, Ceramics & Concrete","glass, ceramics & concrete"


In [45]:
company_industry_df_extended = company_industry_df.assign(**company_industry_df['industry_short_name_lower'].apply(lambda x: get_match_candidates(x, gics_uniq_sectors)))

In [48]:
# company_industry_df_extended.to_csv('gics_candidates_manual_check.csv', index=False)

In [64]:
company_industry_df_extended = pd.read_csv('./gics_candidates_manual_check.csv')

In [65]:
NEW_MANUAL_CATEGORIES = [
    'other',
    'government agency',
    'nonprofit'
]

In [66]:
def get_mapped_level1_sector(manual_match_value, gics_desc_level1_map):
    if manual_match_value in NEW_MANUAL_CATEGORIES:
        return manual_match_value
    
    level1_sector = gics_desc_level1_map[manual_match_value]
    return level1_sector

In [68]:
company_industry_df_extended['manual_match_level1_sector'] = company_industry_df_extended['manual_match'].apply(lambda x: get_mapped_level1_sector(x, gics_desc_level1_map))

In [70]:
company_industry_df_extended['manual_match_level1_sector'].value_counts()

other                         30
consumer discretionary        25
industrials                   19
information technology         7
financials                     5
health care                    5
consumer staples               5
materials                      3
real estate                    2
government agency              2
telecommunication services     2
energy                         1
nonprofit                      1
utilities                      1
Name: manual_match_level1_sector, dtype: int64

In [72]:
company_industry_df_extended.to_csv('industry_to_gics_sector_map.csv', index=False)